**BERT 모델 불러오기**

In [35]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : 알 수 없는 단어에 대한 토큰
- [SEP] : 문장 구분 토큰
- [PAD] : 패딩 토큰
- [CLS] : 문장 시작을 의미하는 토큰
- [MASK] : 마스킹된 단어에 대한 토큰

In [36]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요

def get_special_tokens(tokenizer):
    special_tokens = {
        'unk_token': tokenizer.unk_token,
        'sep_token': tokenizer.sep_token,
        'pad_token': tokenizer.pad_token,
        'cls_token': tokenizer.cls_token,
        'mask_token': tokenizer.mask_token
    }
    return special_tokens

special_tokens = get_special_tokens(tokenizer)
print(special_tokens)

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [37]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt")

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : [CLS] 토큰은 문장의 시작을 나타내기 위해 사용된다. 따라서 입력 시퀀스의 첫 번째에 항상 추가되기 때문에 추가되었다.
- [SEP] 토큰이 추가된 이유 : [SEP] 토큰은 두 문장 간의 구분을 나타내기 위해 사용된다. 또한 문장 쌍 태스크가 아닌 경우에도, 입력 시퀀스의 끝을 나타내는 데 사용되기 때문에 추가되었다.

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [38]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)

In [57]:
# sentence에 대해 BERT를 이용해 감성분류 하는 함수
def sentiment_analysis(sentence):

    encoding = tokenizer(sentence, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.to(device)

    for k,v in encoding.items():
        encoding[k] = v.to(device)

    outputs = model(**encoding)

    prediction_probs = softmax(outputs.logits, dim=1)
    probs_list = prediction_probs.squeeze().tolist()

    # 클래스 레이블 정의
    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    # 결과 출력 (소수점 2자리로 출력)
    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")

**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [58]:
# 매우 부정적인 문장 작성
sentence = "This is an absolute catastrophe. Everything about this experience has been a nightmare from start to finish. The service was a complete disaster, the product is worthless and malfunctioning, and I am left feeling utterly betrayed and infuriated. I can't believe how horrendous this has been; it's the worst possible outcome imaginable."

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.59
부정적일 확률: 0.25
중립적일 확률: 0.11
긍정적일 확률: 0.03
매우 긍정적일 확률: 0.02


In [59]:
# 부정적인 문장 작성
sentence = "The product was disappointing and didn't work as intended. While it was somewhat functional, it failed to meet my expectations and left me dissatisfied."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.27
부정적일 확률: 0.31
중립적일 확률: 0.28
긍정적일 확률: 0.10
매우 긍정적일 확률: 0.03


In [65]:
# 중립적 문장 작성
sentence = "He has a job in the medical field."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.06
부정적일 확률: 0.12
중립적일 확률: 0.33
긍정적일 확률: 0.30
매우 긍정적일 확률: 0.19


In [79]:
# 긍정적 문장 작성
sentence = "Being out feels good."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.11
부정적일 확률: 0.15
중립적일 확률: 0.26
긍정적일 확률: 0.27
매우 긍정적일 확률: 0.21


In [73]:
# 매우 긍정적인 문장 작성
sentence = "She feels happy because the weather is so nice."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.08
부정적일 확률: 0.09
중립적일 확률: 0.19
긍정적일 확률: 0.28
매우 긍정적일 확률: 0.37


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 :

나는 BERT가 주로 문맥적 의미와 단어 간의 관계를 기반으로 감성을 분류한다고 생각한다. BERT는 사전 학습된 언어 모델로서, 문장 내 단어들의 의미를 이해하기 위해 양방향 문맥을 활용한다. 문장의 모든 단어가 문맥에서 어떻게 사용되는지를 고려하여 감성 분류를 수행하는 것이다. 이러한 접근 방식은 문장의 감정을 판단하는 데 중요한 요소인 감정적 뉘앙스, 문맥적 상관관계, 그리고 문장 내 단어들의 상호작용을 보다 정교하게 파악할 수 있게 한다.